Laurie Stevens & Guillaume Palumbo <br>
Année scolaire 2016-2017 <br>
Coding assignment: Traffic Flow <br>

Ce code a pour but de résoudre une équation différentielle, qui se trouve être une équation de flux, décrivant la progression de la vitesse sur un tronçon de route dans le temps et l'espace (par espace, nous entendons ici tous les points du tronçon) selon les variations de la "densité" de voitures présentes sur le tronçon en question. <br>
Le but final du code est donc de sortir les différentes vitesses pour chaque moment et à chaque point de l'espace pour ces temps. <br>
Pour ce faire, il nous faudra d'abord trouver les valeurs de la densité en chaque point, vu que la vitesse se calcule directement à partir de la densité. <br>
Nous calculerons les différentes valeurs de la densité en discrétisant l'équation différentielle qui nous est donnée, avec une différence "forward" dans le temps et "backward" dans l'espace. <br>

Pour commencer, retrons simplement les valeurs initiales requises pour la résolution de ce problème: la vitesse maximale sur le tronçon de route, la longueur du tronçon, la densité maximale de voiture, le nombre d'intervalles dans l'espace (et donc le nombre de points où nous pourrons mesurer la densité, et ainsi la vitesse), le pas utilisé pour le temps et enfin la densité initiale (en x0 pour tous les temps t, et en t0 pour tous les x). <br>
NB: Les valeurs que nous avons laissées pour la vitesse maximale et pour les valeurs initiales de la densité sont celles de la partie B du problème donné sur le site du cours. <br>

In [1]:
import numpy as np

MAX_VEL = 37.78 #Maximum velocity in m/s
LENGTH = 11000.0 #Length in m
MAX_DEN = 250.0 #Rho max
INTERVAL = 51 #nx
TIMELAPS = 3.6 #Timelaps in s
RHO0 = 20.0 #Density's initial value for t=0
INITIAL_MULT = 20.0 #Multiplier for the density array

density_space_array = np.ones(INTERVAL)*INITIAL_MULT
density_space_array[10:20] = 50 #Initial condition

Implémentons maintenant la fonction "flux" qui est donnée dans l'énoncé et qui donne le flux en fonction de la vitesse et de la densité. <br>

In [2]:
def flux(rho):
	"""
	Calculate the flow for a given rho.
	"""
	return MAX_VEL*rho*(1-(rho/MAX_DEN))

Définissons maintenant une fonction "density" qui va se charger de calculer la densité en chaque point de l'espace et du temps. <br>
Nous avons choisi de ranger ces valeurs dans une matrice, où chaque colonne représente une position dans l'espace et chaque ligne une position dans le temps. <br>
Les valeurs sont calculées par discrétisation de l'équation différentielle qui nous est donnée, à partir des valeurs qui nous sont données (c'est à dire la première ligne et la première colonne). La méthode de disrétisation va nous permettre de remplir tous les emplacements vides de cette matrice. <br>
Nous obtenons ainsi une valeur de la densité pour chaque point qui nous intéresse dans l'espace-temps. <br>

In [3]:
def density(density_space_array, flux, time_max):
	"""
	Calculate a density_matrix from a given array and the flow function until time_max.
	"""
	density_matrix=[density_space_array]
	for i in range(1,int(time_max/TIMELAPS)):
		temp_array=[RHO0]
		for j in range(1,INTERVAL):
			temp_array.append(density_matrix[i-1][j]+(TIMELAPS/(LENGTH/INTERVAL))*(flux(density_matrix[i-1][j-1])-flux(density_matrix[i-1][j])))
		density_matrix.append(temp_array)
	return density_matrix

A partir de cette matrice contenant toutes les valeurs de la densité, nous allons calculer les valeurs de la vitesse corrrespondantes, que nous allons également ranger dans une matrice dont les lignes représenteront les différents temps et dont les colonnes représenteront les différents points de l'espace. <br>

In [4]:
def velocity(density_matrix):
	"""
	Calculate a velocity matrix from a given density_matrix.
	"""
	velocity_matrix=[]
	for i in range(len(density_matrix)):
		temp_array=[]
		for j in range(len(density_matrix[i])):
			temp_array.append(MAX_VEL*(1-(density_matrix[i][j]/MAX_DEN)))
		velocity_matrix.append(temp_array)
	return velocity_matrix

Nous voulons que le code sorte des valeurs un peu plus intéressantes qu'une simple liste des valeurs de la densité en chaque point x pour le temps choisi, comme par exemple la valeur moyenne, la valeur la plus petite et la valeur la plus grande de la vitesse pour chaque temps. <br>
Nous allons donc maintenant définir une fonction qui calcule la valeur moyenne de la vitesse pour chaque temps, et donc la valeur moyenne des valeurs de chaque ligne de la matrice. <br>
Les fonctions qui servent à trouver la valeur minimale et la valeur maximale existent déjà en python, il n'y a donc pas besoin de les implémenter ici. <br>

In [5]:
def find_average(array):
	"""
	Calculate the average value of an array.
	"""
	total = 0
	for i in range(len(array)):
		total+=array[i]
	return total/len(array)

Implémentons maintenant la partie principale du code. <br>
Après que le temps souhaité aie été choisi, elle donne le choix entre plusieurs modes d'utilisation: un où on nous donne toutes les valeurs de la vitesse pour le temps demandé, et un autre où on nous donne uniquement la valeur maximale, la valeur minimale et la valeur moyenne pour le temps demandé. <br>
NB: si le temps demandé n'est pas un multiple du pas qui a été implémenté, la valeur qui sortira correspondra au temps multiple du pas précédant le temps qui a été demandé. <br>

In [6]:
if __name__=="__main__":
	time_max=float(input("Quelle valeur de temps final désirez-vous? (en minutes) "))
	mode=int(input("Quel mode souhaitez-vous? 1 : Afficher toutes les valeurs de la vitesse en chaque point pour le temps donné, 2 : Afficher les valeurs minimales, moyennes et maximales pour le temps donné, 0 : Quitter "))
	while mode!=0: #Main loop
		density_matrix=density(density_space_array,flux,time_max*60.0)
		velocity_matrix=velocity(density_matrix)
		if mode == 1 : #The whole array
			print(velocity_matrix[int(time_max*60.0/TIMELAPS)-1])
		elif mode == 2 : #Significant values
			print("La valeur moyenne est : "+str(find_average(velocity_matrix[int(time_max*60.0/TIMELAPS)-1]))+" m/s")
			print("La valeur minimale est : "+str(min(velocity_matrix[int(time_max*60.0/TIMELAPS)-1]))+" m/s")
			print("La valeur maximale est : "+str(max(velocity_matrix[int(time_max*60.0/TIMELAPS)-1]))+" m/s")
		else: #Wrong input
			print("Veuillez entrer 1, 2 ou 0")
		time_max=float(input("Quelle valeur de temps final désirez-vous? (en minutes) "))
		mode=int(input("Quel mode souhaitez-vous? 1 : Afficher toutes les valeurs de la vitesse en chaque point pour le temps donné, 2 : Afficher les valeurs minimales, moyennes et maximales pour le temps donné, 0 : Quitter "))
	print("Au revoir!")

Quelle valeur de temps final désirez-vous? (en minutes) 3
Quel mode souhaitez-vous? 1 : Afficher toutes les valeurs de la vitesse en chaque point pour le temps donné, 2 : Afficher les valeurs minimales, moyennes et maximales pour le temps donné, 0 : Quitter 2
La valeur moyenne est : 33.8738780716 m/s
La valeur minimale est : 30.9720006425 m/s
La valeur maximale est : 34.757600000000004 m/s
Quelle valeur de temps final désirez-vous? (en minutes) 0
Quel mode souhaitez-vous? 1 : Afficher toutes les valeurs de la vitesse en chaque point pour le temps donné, 2 : Afficher les valeurs minimales, moyennes et maximales pour le temps donné, 0 : Quitter 0
Au revoir!


Les valeurs qui sont obtenues paraissent toutes cohérentes et ont été vérifiées sur le site du cours (et ainsi confirmées). <br>